In [1]:
import os
import numpy as np
import pandas as pd

In [ ]:
y = pd.read_table(os.path.join('data', 'data_output.txt'),sep='\s+', engine='python', dtype ='Int64')
X = pd.read_table(os.path.join('data', 'data_input.txt'),sep='\s+', engine='python', dtype ="Float64")

In [ ]:
ind1_list = []
ind2_list = []
for i in range(X.shape[0]):
    ind1_list.append(i//4) 
    ind2_list.append(i%4+1)
mInd = pd.MultiIndex.from_arrays([ind1_list, ind2_list], names=('number', 'mode'))
X.index = mInd

In [ ]:
X = np.abs(X.copy())
n1 = X.columns[1:9:2]
n2 = X.columns[2:9:2]
formula = n1 + n2 + ' = (' + n1 + ' + ' + n2 + ')/2'

In [ ]:
def norm_frez(x):
    #x['frez'].iloc[3] = x['frez'].iloc[3] - 14.7  
    f_deltamin = x['frez'] - np.min(x['frez'])
    f_norm = f_deltamin/np.max(f_deltamin)
    x['frez'] = f_norm.round(3)
    return x  

In [ ]:
X = X.groupby(level = 'number').apply(norm_frez)

In [ ]:
for i in range(4): X = X.eval(formula[i]).drop(columns = [n1[i],n2[i]]) 
X_frez = X.pop('frez')
maxval = np.max(X, axis = 1)
X = X.divide(maxval, axis = 0, level=1)
X = X.round(3)
X.insert(0,'frez', X_frez)

In [ ]:
ind = []
for i in range(X.index[-1][0]):
    Xstart = X.loc[i].drop(columns = 'frez')
    for j in (range((i + 1),X.index[-1][0])):
        Xnext = X.loc[j].drop(columns = 'frez')
        if (np.mean(np.abs(Xnext - Xstart), axis = 1) < 0.05).all():
            ind.append([i,j])  